In [37]:
from elasticsearch import Elasticsearch
from elasticsearch import NotFoundError
import boto3

In [42]:
es = Elasticsearch("http://elasticsearch:9200")
INDEX_NAME = "course-questions"

In [40]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_row = docs_response.json()

documents_row[0]["course"]

'data-engineering-zoomcamp'

In [41]:
import hashlib

documents = []

for course in documents_row:

    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        question_hash = hashlib.md5(doc["question"].encode()).hexdigest()
        doc["id"] = question_hash
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': '2d669d12c0511996b393bff34bfbcf55'}

In [45]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "id": {"type": "keyword"},
            "course": {"type": "keyword"},
        }
    },
}

try:
    es.indices.get(index=INDEX_NAME)
    print(f"{INDEX_NAME} already exists")
    # es.indeces.delete(index=INDEX_NAME, ignore=[400, 404])
except NotFoundError:
    response = es.indices.create(index=INDEX_NAME, body=index_settings)
    print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'}


In [48]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=INDEX_NAME, id=doc["id"], document=doc)

count_response = es.count(index=INDEX_NAME)
print(count_response)

100%|██████████| 948/948 [00:03<00:00, 248.82it/s]


{'count': 943, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
